In [1]:
# Load pickled data
import matplotlib.pyplot as plt
import pickle, cv2, csv, os, time
import numpy as np
from numpy import newaxis
from random import randint
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import gridspec
%matplotlib inline
from keras.models import model_from_json
from keras.optimizers import Adam
import copy
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras.regularizers import l2
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from skimage.transform import rescale, resize, downscale_local_mean

from PIL import Image

Using TensorFlow backend.


In [4]:


current_path = os.getcwd()
print (current_path)
train_path1 = current_path + "/train/Type_1/"
train_path2 = current_path + "/train/Type_2/"
train_path3 = current_path + "/train/Type_3/"
add_type1 = current_path + "/additional_Type_1_v2/Type_1/" 
add_type2 = current_path + "/additional_Type_2_v2/Type_2/" 
add_type3 = current_path + "/additional_Type_3_v2/Type_3/" 

test = current_path+"test/"

path_list =[]
path_list.append(add_type1)
path_list.append(add_type2)
path_list.append(add_type3)
path_list.append(train_path1)
path_list.append(train_path2)
path_list.append(train_path3)

path_list.append(test)

 
def resize_images(path,pickle_filename):
    resize_img=[]
    file_names = [] #for test file names for submission
    size_dict= {}
    #for path in path_list:
    start_time = time.time()
    for root, dirs, files in os.walk(path, topdown=False):
        print ("path:",path,' numFiles:',len(files))
        for f in files:
            if f !='.DS_Store':
                print ("path:",path," path+f:",path+f)
                img = cv2.imread(path+f)
                print ("img shape:",img.shape)
                image_resize = resize(img, (224, 224), mode='reflect')
                print("img_resize shape:",image_resize.shape)
                resize_img.append(image_resize)
                file_names.append(f)
                print("size list:",len(resize_img))                
                size_dict[img.shape[0]]=img.shape[1]
        print("dict of image dimensions:",size_dict)
        if(pickle_filename=='test.p'):
            with open('test_filename.p','wb') as save_file:
                pickle.dump(file_names,save_file)
        with open(pickle_filename,'wb') as save_file:
            pickle.dump(resize_img,save_file)
    end_time = time.time()
    print("elapsed time:",(end_time-start_time)/60.)

#uncomment each one to run. Should run wo memory error. 
#resize_images(train_path1,'type1_resize.p')
#resize_images(train_path2,'type2resize.p')
#resize_images(train_path3,'type3resize.p')
#resize_images(add_type1,'type1addresize.p')
resize_images(test,'test.p')

            

/home/dc/DeepLearning/kaggle
path: /home/dc/DeepLearning/kaggle/test/  numFiles: 512
path: /home/dc/DeepLearning/kaggle/test/  path+f: /home/dc/DeepLearning/kaggle/test/397.jpg
img shape: (4128, 3096, 3)
img_resize shape: (224, 224, 3)
size list: 1
path: /home/dc/DeepLearning/kaggle/test/  path+f: /home/dc/DeepLearning/kaggle/test/163.jpg
img shape: (3264, 2448, 3)
img_resize shape: (224, 224, 3)
size list: 2
path: /home/dc/DeepLearning/kaggle/test/  path+f: /home/dc/DeepLearning/kaggle/test/72.jpg
img shape: (4128, 3096, 3)
img_resize shape: (224, 224, 3)
size list: 3
path: /home/dc/DeepLearning/kaggle/test/  path+f: /home/dc/DeepLearning/kaggle/test/496.jpg
img shape: (3264, 2448, 3)
img_resize shape: (224, 224, 3)
size list: 4
path: /home/dc/DeepLearning/kaggle/test/  path+f: /home/dc/DeepLearning/kaggle/test/482.jpg
img shape: (4128, 3096, 3)
img_resize shape: (224, 224, 3)
size list: 5
path: /home/dc/DeepLearning/kaggle/test/  path+f: /home/dc/DeepLearning/kaggle/test/425.jpg
img 

In [2]:
img1 = []
read_file = open('type1_resize.p', 'rb') 
imgs_type1 = pickle.load( read_file)
img1 += imgs_type1
print("num type1 images(250):", len(imgs_type1)) #should be 250

read_file = open('type1add_resize.p', 'rb') 
imgs_type1 = pickle.load( read_file)
img1 += imgs_type1
print('num type1 additional images(1190):',len(imgs_type1))


print(len(img1))
label_1 = [0]*(len(img1))
print ('sample label for class Type1 should be 0:',label_1[0])
print ('num labels_1:',len(label_1))

num type1 images(250): 250
num type1 additional images(1190): 1190
1440
sample label for class Type1 should be 0: 0
num labels_1: 1440


In [7]:
read_file = open('test.p','rb')
imgs_test = pickle.load(read_file)
print(len(imgs_test))

512


In [8]:


read_file = open('test_filename.p', 'rb')
test_file_names = pickle.load(read_file)
print (len(test_file_names))

512


In [ ]:
path = '/Users/dc/DeepLearning/kaggle/additional_Type_2_v2/Type_2/'
path_first='/Users/dc/DeepLearning/kaggle/first1k/'
path_second='/Users/dc/DeepLearning/kaggle/2ndk/'
path_third='/Users/dc/DeepLearning/kaggle/3rdk/'
num=0
for root, dirs, files in os.walk(path):
    for file in files:
        print('moving:',file)
        if (file != '.DS_Store'):
            if (num < 1000):
                os.rename(path+file,path_first+file)
            elif ((num>1000) and (num<2000)):
                os.rename(path+file,path_second+file)
            else:
                os.rename(path+file,path_third+file)
            num+=1
    print('num:',num)

In [ ]:
path_first='/Users/dc/DeepLearning/kaggle/first1k/'
path_second='/Users/dc/DeepLearning/kaggle/2ndk/'
path_third='/Users/dc/DeepLearning/kaggle/3rdk/'

resize_images(path_first,'type2_first.p')
resize_images(path_second,'type2_second.p')
resize_images(path_third,'type2_third.p')


In [ ]:
#redo type 2. Too many files for stupid pickle format

path = '/Users/dc/DeepLearning/kaggle/additional_Type_2_v2'
first1k=[]
second1k=[]
third1k=[]
for file in os.walk(path):
    print (file)
    img = cv2.imread(path+file)
    image_resize = resize(img, (224, 224), mode='reflect')
    if (num < 1000) and (file!='.DS_Store'):
        first1k.append(image_resize)
    else if ((num>1000) and (num<2000)):
        second1k.append(image_resize)
    else:
        thirdk.append(image_resize)
    num+=1


In [3]:
#redo type2

img2 = []

read_file = open('type2resize.p', 'rb') 
imgs_type2 = pickle.load( read_file)
img2 += imgs_type2
print("num type2_resize:", len(imgs_type2)) #should be 1000-1 for DS_Store


read_file = open('type2_first.p', 'rb') 
imgs_type2 = pickle.load( read_file)
img2 += imgs_type2
print("num type2_first:", len(imgs_type2)) #should be 1000

read_file = open('type2_second.p', 'rb') 
imgs_type2 = pickle.load( read_file)
img2 += imgs_type2
print("num type2_second:", len(imgs_type2)) #should be 999

read_file = open('type2_third.p', 'rb') 
imgs_type2 = pickle.load( read_file)
img2 += imgs_type2
print("num type2_third:", len(imgs_type2)) #should be 1566


label_2=[1]*(781+1000+999+1566)
print ('label_2:',len(label_2))
print ('sample label for Type2 should be 1:',label_2[0])
print ('num img2:',len(img2))


num type2_resize: 781
num type2_first: 1000
num type2_second: 999
num type2_third: 1566
label_2: 4346
sample label for Type2 should be 1: 1
num img2: 4346


In [ ]:
# redo type3 move type3 files
path = '/Users/dc/DeepLearning/kaggle/additional_Type_3_v2/Type_3/'
path_first='/Users/dc/DeepLearning/kaggle/first_third1k/'
path_second='/Users/dc/DeepLearning/kaggle/third2ndk/'
#path_third='/Users/dc/DeepLearning/kaggle/third3rdk/'

num = 0

for root, dirs, files in os.walk(path):
    for file in files:
        print('moving:',file)
        if (file != '.DS_Store'):
            if (num < 1000):
                os.rename(path+file,path_first+file)
            elif ((num>1000) and (num<2000)):
                os.rename(path+file,path_second+file)
            else:
                os.rename(path+file,path_third+file)
            num+=1
    print('num:',num)

In [ ]:
path_first='/Users/dc/DeepLearning/kaggle/first_third1k/'
path_second='/Users/dc/DeepLearning/kaggle/third2ndk/'
#path_third='/Users/dc/DeepLearning/kaggle/3rdk/'

resize_images(path_first,'type3_first.p')
resize_images(path_second,'type3_second.p')


In [4]:
img3=[]

read_file = open('type3resize.p', 'rb') 
imgs_type3 = pickle.load( read_file)
img3 += imgs_type3
print("num type3_resize:", len(imgs_type3)) #should be 


read_file = open('type3_first.p', 'rb') 
imgs_type3 = pickle.load( read_file)
img3 += imgs_type3
print("num type3_first:", len(imgs_type3)) #should be 1000

read_file = open('type3_second.p', 'rb') 
imgs_type3 = pickle.load( read_file)
img3 += imgs_type3
print("num type3_second:", len(imgs_type3)) #should be 975


print('num img3:',len(img3))
label_3 = [2]*(450+1000+975)
print ('len label_3:', len(label_3))
print ('sample label for Type3, should be 2:',label_3[0])

#convert after you mix them up and shuffle
#label_3add = np_utils.to_categorical(label_3add, 3)


num type3_resize: 450
num type3_first: 1000
num type3_second: 975
num img3: 2425
len label_3: 2425
sample label for Type3, should be 2: 2


In [12]:
#error check
img = img1+img2+img3
label=label_1+label_2+label_3
print(len(img), len(label))
print(type(img),type(label))
X_train = np.array(img)
print ('max:',np.max(X_train[0]))
print ('min:',np.min(X_train[0]))
print (X_train[100])
print (np.mean(X_train))
#X_train = X_train/255.
y_train = np_utils.to_categorical(label, 3)
print(X_train.shape, y_train.shape)
X_train,y_train=shuffle(X_train,y_train)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train,y_train, test_size=.20, random_state=42)

8211 8211
<class 'list'> <class 'list'>
max: 1.0
min: 0.0
[[[ 0.69019608  0.54117647  0.77647059]
  [ 0.68261305  0.53359344  0.76888756]
  [ 0.68241297  0.52947179  0.77260904]
  ..., 
  [ 0.52262905  0.35029012  0.61261505]
  [ 0.50518207  0.34439776  0.58753501]
  [ 0.51652661  0.3557423   0.59495798]]

 [[ 0.68772509  0.52694078  0.77007803]
  [ 0.68222289  0.52928171  0.77241897]
  [ 0.6752501   0.52230892  0.76544618]
  ..., 
  [ 0.52253902  0.35783313  0.61273509]
  [ 0.51518607  0.35048019  0.60538215]
  [ 0.49217687  0.33419368  0.58909564]]

 [[ 0.68459384  0.52380952  0.76694678]
  [ 0.66407563  0.51113445  0.75427171]
  [ 0.68055222  0.52761104  0.7707483 ]
  ..., 
  [ 0.53067227  0.35833333  0.62065826]
  [ 0.51355542  0.34492797  0.60767307]
  [ 0.5182473   0.3510004   0.6137455 ]]

 ..., 
 [[ 0.2009904   0.16569628  0.36269508]
  [ 0.21935774  0.18406363  0.38406363]
  [ 0.21995798  0.17682073  0.39642857]
  ..., 
  [ 0.55854342  0.45658263  0.69187675]
  [ 0.56453581  0

In [9]:
X_test = np.array(imgs_test)

In [10]:
print(X_test.shape)

(512, 224, 224, 3)


In [10]:
asdf=np_utils.to_categorical(y_train,3)
asdf[0]

array([ 0.,  1.,  0.])

In [ ]:
import scipy.ndimage as ndi

#crop test
img = cv2.imread('/Users/dc/DeepLearning/kaggle/train/Type_1/0.jpg')

height = img.shape[0]
width = img.shape[1]
cv2.line(img,(0,int(height/2.)),(int(width),int(height/2.0)),(255,0,0))
cv2.line(img,(int(width/2.),0),(int(width/2.0),height),(255,0,0))

trans_factor=.8 
zoom_range=[.8,.8]


y_offset = np.random.uniform(0,trans_factor*width)
x_offset = np.random.uniform(0,trans_factor*height)

print ('y_offset:', y_offset, "x_offset:", x_offset)

offset = np.array([0,200])
#center image
#offset=np.array([10,10])
scale_factor = np.random.uniform(zoom_range[0],zoom_range[1])
print (scale_factor)
crop = np.array([[scale_factor,0],[0,scale_factor]])

print('image.shape before roll:',img.shape)
img_roll = np.rollaxis(img,axis=-1, start=0)
print('image.shape after roll:',img_roll.shape)
#this is for the affine transform for each channel you have to roll the axis? Seems funky
#you can use a numpy.reshape also. this is clearer and more mainstream

image_channel = [ndi.interpolation.affine_transform(image_channel,
                        crop, offset=offset, order=0, mode='nearest',
                        cval=0.0) for image_channel in img_roll]
image_array = np.stack(image_channel, axis=0)
image_array = np.rollaxis(image_array, 0, 3)

print ('image_array:',image_array.shape)
#height_rs = image_array.shape[0]
#width_rs = image_array.shape[1]
#print(height_rs, width_rs)
#cv2.line(image_array,(10,10),(100,100),(255,0,0))
#cv2.line(image_array,(0,int(height_rs/2.)),(int(width_rs),int(height_rs/2.0)),(255,0,0))
#cv2.line(image_array,(int(width_rs/2.),0),(int(width_rs/2.0),height_rs),(255,0,0))

fig = plt.figure()

ax = fig.add_subplot(2, 2, 1)
ax.imshow(img)
plt.title("orig")
ax1 = fig.add_subplot(2, 2, 2)
ax1.imshow(image_array)
plt.title("crop .8")
#fig.add_subplot(image_array)

#plt.show()
#problem is the shift away from center in the crop .8 image

In [ ]:
#we can crop to produce a smaller resolution. 
import skimage
from skimage.transform import rescale, resize, downscale_local_mean

image = cv2.imread('/Users/dc/DeepLearning/kaggle/train/Type_1/0.jpg')
height = image.shape[0]
width = image.shape[1]
cv2.line(image,(0,int(height/2.)),(int(width),int(height/2.0)),(255,0,0))
cv2.line(image,(int(width/2.),0),(int(width/2.0),height),(255,0,0))


print ('orig:',image.shape)
image_resized = resize(image, (400, 400), mode='reflect')
height = image_resized.shape[0]
width = image_resized.shape[1]
cv2.line(image_resized,(0,int(height/2.)),(int(width),int(height/2.0)),(255,0,0))
cv2.line(image_resized,(int(width/2.),0),(int(width/2.0),height),(255,0,0))

#image_downscaled = downscale_local_mean(image, (480, 640))
print ('resized:',image_resized.shape)
fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True)

ax = axes.ravel()
ax[0].imshow(image)
ax[0].set_title("Original image")

ax[1].imshow(image_resized)
ax[1].set_title("resized")

#ax[2].imshow(image_downscaled)
#ax[2].set_title("downscaled")


In [ ]:
import os,random


def move_file(source_path,dest_path):
    """
    input: source path, dest path
    output: move 20 percent files from source path to dest path
    """
    list_files = os.listdir(source_path)
    print (len(list_files))

    num_moved=0
    for x in range(1,int(.20*len(list_files))):
        index = random.randint(1, len(list_files))
        print(index,list_files[index])
        source_file = source_path+list_files[index]
        if(os.path.isfile(source_file)):
            dest_file = dest_path+list_files[index]
            os.rename(source_file,dest_file)
            num_moved+=1

    print (num_moved)

#move_file(os.getcwd()+"/data/train/type1/",os.getcwd()+"/data/valid/type1/")
#move_file(os.getcwd()+"/data/train/type2/",os.getcwd()+"/data/valid/type2/")
move_file(os.getcwd()+"/data/train/type3/",os.getcwd()+"/data/valid/type3/")


In [10]:
import os
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam


# number of classes
nb_classes = 3

# input image dimensions
img_rows, img_cols,img_ch = 224, 224, 3

# number of convolutional filters to use
nb_filter1 = 32
nb_filter2 = 64
nb_filter3 = 128



model = Sequential()
model.add(Convolution2D(32, kernel_size=(3, 3),padding='same',
                        input_shape=(img_rows,img_cols,img_ch),name='conv1'))
model.add(Activation('relu',name='relu1'))
model.add(MaxPooling2D(pool_size=(2,2),strides=1,name='maxpool1'))
model.add(Convolution2D(64, kernel_size=(3, 3),padding='same',
                        name='conv2'))
model.add(Activation('relu',name='relu2'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None,name='maxpool2'))
model.add(Convolution2D(128, kernel_size=(3, 3), padding='same',
                        name='conv3'))
model.add(Activation('relu',name='relu3'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None,name='maxpool3'))
model.add(Flatten(name='flatten'))
model.add(Dropout(0.5,name='dropout1'))
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu',name='relu4'))
model.add(Dropout(0.5,name='dropout2'))
model.add(Dense(128,  name='hidden2'))
model.add(Activation('relu',name='relu5'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 224, 224, 32)      896       
_________________________________________________________________
relu1 (Activation)           (None, 224, 224, 32)      0         
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 223, 223, 32)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 223, 223, 64)      18496     
_________________________________________________________________
relu2 (Activation)           (None, 223, 223, 64)      0         
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 111, 111, 64)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 111, 111, 128)     73856     
__________

In [ ]:
import time
nb_epoch = 1000
batch_size=50

model_name='test.chkpt'

if os.path.isfile(model_name):
    try:
        model = load_model(model_name)
        print('loading trained model ...')
    except Exception as e:
        print('Unable to load model', model_name, ':', e)
        raise    


adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

startTime = time.time()
history = model.fit(X_train1, y_train1,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=2, validation_data=(X_test1, y_test1))

endTime = time.time()
print('elapsed secs:', (endTime - startTime)/1000.)
#nvidia-smi to see power stats nvidia-smi -i 3 -l -q -d

Train on 6568 samples, validate on 1643 samples
Epoch 1/1000
45s - loss: 0.8896 - acc: 0.5726 - val_loss: 0.8466 - val_acc: 0.6190
Epoch 2/1000
44s - loss: 0.8649 - acc: 0.5868 - val_loss: 0.8301 - val_acc: 0.6202
Epoch 3/1000
44s - loss: 0.8353 - acc: 0.5964 - val_loss: 0.8252 - val_acc: 0.6093
Epoch 4/1000
45s - loss: 0.8090 - acc: 0.6192 - val_loss: 0.8006 - val_acc: 0.6220
Epoch 5/1000
45s - loss: 0.7729 - acc: 0.6366 - val_loss: 0.7802 - val_acc: 0.6385
Epoch 6/1000
45s - loss: 0.7365 - acc: 0.6586 - val_loss: 0.7840 - val_acc: 0.6306
Epoch 7/1000
46s - loss: 0.7030 - acc: 0.6755 - val_loss: 0.7743 - val_acc: 0.6500
Epoch 8/1000
46s - loss: 0.6513 - acc: 0.7083 - val_loss: 0.8366 - val_acc: 0.6306
Epoch 9/1000
45s - loss: 0.6177 - acc: 0.7188 - val_loss: 0.7604 - val_acc: 0.6598
Epoch 10/1000
45s - loss: 0.5714 - acc: 0.7483 - val_loss: 0.7834 - val_acc: 0.6598
Epoch 11/1000
45s - loss: 0.5255 - acc: 0.7658 - val_loss: 0.8407 - val_acc: 0.6403
Epoch 12/1000
45s - loss: 0.4874 - ac

In [8]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [6]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
loaded_model.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 224, 224, 32)      896       
_________________________________________________________________
relu1 (Activation)           (None, 224, 224, 32)      0         
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 223, 223, 32)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 223, 223, 64)      18496     
_________________________________________________________________
relu2 (Activation)           (None, 223, 223, 64)      0         
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 111, 111, 64)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 111, 111, 128)   

In [10]:
#this doesnt work to reset
from keras import backend as K

def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

limit_mem()

In [ ]:
type(train_datagen)

In [11]:

predict = loaded_model.predict(X_test)

In [23]:

with open('result.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    
    for i in range(0,len(predict)):
        print(test_file_names[i],predict[i])
        writer.writerow((test_file_names[i],predict[i][0],predict[i][1], predict[i][2]))
    writer.close()

397.jpg [  1.00749046e-13   1.00000000e+00   2.02010197e-11]
163.jpg [  4.01922443e-06   9.99995947e-01   1.94054532e-08]
72.jpg [  1.95965126e-17   1.00000000e+00   1.15588389e-18]
496.jpg [  2.67387688e-04   1.17290035e-01   8.82442594e-01]
482.jpg [  1.00000000e+00   1.74733249e-11   2.29009714e-15]
425.jpg [  5.31143966e-12   5.17032295e-03   9.94829714e-01]
344.jpg [ 0.94375873  0.03356249  0.02267887]
29.jpg [  2.91655450e-10   9.77435350e-01   2.25646850e-02]
241.jpg [  1.39422714e-15   1.00000000e+00   9.16897863e-19]
258.jpg [  6.53213421e-12   1.40599981e-02   9.85939980e-01]
467.jpg [  9.99999642e-01   3.22680791e-07   9.48221016e-12]
324.jpg [ 0.05689519  0.84923977  0.09386502]
415.jpg [  1.00000000e+00   1.30851303e-13   9.83321144e-17]
47.jpg [  9.97296393e-01   2.70362291e-03   3.14118245e-11]
352.jpg [  2.95254431e-05   1.36785442e-04   9.99833703e-01]
471.jpg [  2.60650273e-03   9.97393489e-01   4.42321769e-11]
107.jpg [  7.90108439e-25   1.00000000e+00   1.66753829e-

AttributeError: '_csv.writer' object has no attribute 'close'